# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [15]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.core.compute import AmlCompute
from azureml.core.compute import ComputeTarget
from azureml.core.compute_target import ComputeTargetException
from azureml.train.automl import AutoMLConfig
from azureml.widgets import RunDetails

## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [16]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'NCAA_matchup_classifier'

experiment=Experiment(ws, experiment_name)

# Choose a name for your CPU cluster
amlcompute_cluster_name = "aml-compute"

try:
    compute_target = ComputeTarget(workspace=ws, name=amlcompute_cluster_name)
    print("Found existing cluster,", amlcompute_cluster_name)
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size='Standard_DS3_v2',
                                                           #vm_priority = 'lowpriority',
                                                           max_nodes=6)
    compute_target = ComputeTarget.create(ws, amlcompute_cluster_name, compute_config)

compute_target.wait_for_completion(show_output=True, min_node_count = 0, timeout_in_minutes = 10)

Found existing cluster, aml-compute
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [6]:
ds = TabularDatasetFactory.from_delimited_files(path="https://s3.amazonaws.com/static.akk.io/Akkio-NCAAHistoricalData.csv")
pdata = ds.to_pandas_dataframe()
#pdata.drop("Column1", inplace=True, axis=1)
#pdata.drop("index", inplace=True, axis=1)
#pdata.drop("Score1", inplace=True, axis=1)
#pdata.drop("Score2", inplace=True, axis=1)
#pdata.drop("spread", inplace=True, axis=1)
pdata = pdata[pdata["Year"]>2009]
#Best VotingEnsemble with all ~50 features had accuracy 0.84921
#Compact to top 10 features, essentially top 5 stats for each team, for a more user friendly API
topTenFeatures = ["G1", "G2", "Seed1", "Seed2", "PF1", "PF2", "3P%1", "3P%2", "TOV1", "TOV2", "winner"]
pdata = pdata[topTenFeatures]
TabularDatasetFactory.register_pandas_dataframe(pdata, ws.get_default_datastore(), name="ncaa_2010-2019_compact")

Method register_pandas_dataframe: This is an experimental method, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.


Validating arguments.
Arguments validated.
Successfully obtained datastore reference and path.
Uploading file to managed-dataset/99911a12-7797-45f2-a425-17f64cb1609c/
Successfully uploaded file to datastore.
Creating and registering a new dataset.
Successfully created and registered a new dataset.


{
  "source": [
    "('workspaceblobstore', 'managed-dataset/99911a12-7797-45f2-a425-17f64cb1609c/')"
  ],
  "definition": [
    "GetDatastoreFiles",
    "ReadParquetFile",
    "DropColumns"
  ],
  "registration": {
    "id": "f6128809-67a2-4c11-94c8-9324efc436cc",
    "name": "ncaa_2010-2019_compact",
    "version": 1,
    "workspace": "Workspace.create(name='quick-starts-ws-149224', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-149224')"
  }
}

## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [19]:
# TODO: Put your automl settings here
automl_settings = {
    "primary_metric" : "accuracy",
    "experiment_timeout_hours": 1.0,
    "max_concurrent_iterations": 5,
    "n_cross_validations": 10
}

# TODO: Put your automl config here
automl_config = AutoMLConfig(compute_target=compute_target,
                             task = "classification",
                             training_data=ws.datasets["ncaa_2010-2019_compact"],
                             label_column_name="winner",
                             featurization= 'auto',
                             **automl_settings
                            )

In [20]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config)

Submitting remote run.


Experiment,Id,Type,Status,Details Page,Docs Page
NCAA_matchup_classifier,AutoML_e42d18ca-46d2-4893-b991-4d1b4d6bf9bd,automl,NotStarted,Link to Azure Machine Learning studio,Link to Documentation


## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [21]:
RunDetails(remote_run).show()

_AutoMLWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', 's…

_UserRunWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO', '…

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [26]:
best_auto_run, best_auto_model = remote_run.get_output()
best_auto_run_metrics = best_auto_run.get_metrics()

print("ID: ", best_auto_run.id)
print("accuracy: ", best_auto_run_metrics["accuracy"])
best_auto_model

Package:azureml-automl-runtime, training version:1.31.0, current version:1.30.0
Package:azureml-core, training version:1.31.0, current version:1.30.0
Package:azureml-dataprep, training version:2.18.0, current version:2.15.1
Package:azureml-dataprep-native, training version:36.0.0, current version:33.0.0
Package:azureml-dataprep-rslex, training version:1.16.0, current version:1.13.0
Package:azureml-dataset-runtime, training version:1.31.0, current version:1.30.0
Package:azureml-defaults, training version:1.31.0, current version:1.30.0
Package:azureml-interpret, training version:1.31.0, current version:1.30.0
Package:azureml-mlflow, training version:1.31.0, current version:1.30.0
Package:azureml-pipeline-core, training version:1.31.0, current version:1.30.0
Package:azureml-telemetry, training version:1.31.0, current version:1.30.0
Package:azureml-train-automl-client, training version:1.31.0, current version:1.30.0
Package:azureml-train-automl-runtime, training version:1.31.0, current ver

ID:  AutoML_e42d18ca-46d2-4893-b991-4d1b4d6bf9bd_175
accuracy:  0.8595238095238097


PipelineWithYTransformations(Pipeline={'memory': None,
                                       'steps': [('datatransformer',
                                                  DataTransformer(enable_dnn=False, enable_feature_sweeping=True, feature_sweeping_config={}, feature_sweeping_timeout=86400, featurization_config=None, force_text_dnn=False, is_cross_validation=True, is_onnx_compatible=False, observer=None, task='classification', working_dir='/mn...
                                                 ('LogisticRegression',
                                                  LogisticRegression(C=1.2067926406393288,
                                                                     class_weight='balanced',
                                                                     dual=False,
                                                                     fit_intercept=True,
                                                                     intercept_scaling=1,
                          

In [30]:
#TODO: Save the best model
best_auto_run.register_model(model_name="automl_best_model", model_path="./outputs/")

Model(workspace=Workspace.create(name='quick-starts-ws-149224', subscription_id='9e65f93e-bdd8-437b-b1e8-0647cd6098f7', resource_group='aml-quickstarts-149224'), name=automl_best_model, id=automl_best_model:3, version=3, tags={}, properties={})

INFO:interpret_community.common.explanation_utils:Using default datastore for uploads


## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service